In [1]:
import requests
import os
import json
import pandas as pd
import time
import pymysql

In [2]:
# Connect to the database
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='root@123',
                             db='mentegy')

In [3]:
# Creating a cursor
cursor = connection.cursor()

In [4]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [5]:
%env BEARER_TOKEN= AAAAAAAAAAAAAAAAAAAAAPJZOAEAAAAA4AHhQ5g%2FgMT%2F8%2FSbrwkaATyPFCc%3D6WAGebh9bWk93cLHkDhmfVIVLiGIHqE8sQ8O2Ry7IjihDPVPis 

env: BEARER_TOKEN=AAAAAAAAAAAAAAAAAAAAAPJZOAEAAAAA4AHhQ5g%2FgMT%2F8%2FSbrwkaATyPFCc%3D6WAGebh9bWk93cLHkDhmfVIVLiGIHqE8sQ8O2Ry7IjihDPVPis


In [6]:
def auth():
    return os.environ.get("BEARER_TOKEN")


def create_url():
    return "https://api.twitter.com/2/tweets/sample/stream"

In [7]:
# To set your enviornment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


In [8]:
def get_rules(headers, bearer_token):
    with requests.get("https://api.twitter.com/2/tweets/search/stream/rules", headers=headers) as response:
        if response.status_code != 200:
            raise Exception(
                "Cannot get rules (HTTP {}): {}".format(response.status_code, response.text)
            )
    #print(json.dumps(response.json()))
    return response.json()
    

In [9]:
def delete_all_rules(headers, bearer_token, rules):
    if rules is None or "data" not in rules:
        return None

    ids = list(map(lambda rule: rule["id"], rules["data"]))
    payload = {"delete": {"ids": ids}}
    with requests.post("https://api.twitter.com/2/tweets/search/stream/rules",headers=headers,json=payload) as response:
        if response.status_code != 200:
            raise Exception(
                "Cannot delete rules (HTTP {}): {}".format(
                    response.status_code, response.text
                )
            )
    #print(json.dumps(response.json()))
    

In [10]:
def set_rules(headers, delete, bearer_token):
    # You can adjust the rules if needed
    sample_rules = [
        {"value": "dog has:images", "tag": "dog pictures"},
        {"value": "cat has:images -grumpy", "tag": "cat pictures"},
    ]
    payload = {"add": sample_rules}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        headers=headers,
        json=payload,
    )
    if response.status_code != 201:
        raise Exception(
            "Cannot add rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    #print(json.dumps(response.json()))


In [11]:
def get_stream(headers, set, bearer_token):
    
    r = ''
    while r == '':
        try:
            with requests.get("https://api.twitter.com/2/tweets/search/stream?tweet.fields=created_at&expansions=author_id&user.fields=created_at", headers=headers, stream=True,) as response:
            #print(response.status_code)
                if response.status_code != 200:
                    raise Exception(
                        "Cannot get stream (HTTP {}): {}".format(
                            response.status_code, response.text
                        )
                    )
                #tweetDict = []
                #dataFrame = pd.DataFrame(columns = ['created_at','compound'])
                for response_line in response.iter_lines():
                    if response_line:
                        json_response = json.loads(response_line)
                        #print(json.dumps(json_response, indent=4, sort_keys=True)) 
                        #print(type(json_response['data']['created_at'])) #class str

                        created_at = pd.to_datetime(json_response['data']['created_at']) #converting to datetime
                        text = json_response['data']['text']
                        ps = analyzer.polarity_scores(text)
                        #print(type(ps))
                        #tweetDict.append({'created_at':created_at,'text': text, 'compound_score':ps['compound']})
                        #dataframe = pd.DataFrame.from_dict(json_response['data'], orient="index")
                        #dataFrame = pd.DataFrame(tweetDict)
                        #print("{}… {}".format(created_at, ps, text))  
                        #dataFrame.to_csv('stream.csv')
                        sql = "INSERT INTO `twitter_stream` (`created_at`, `text`, `compound_score`) VALUES (%s, %s, %s);"
                        cursor.execute(sql,(created_at,text,ps['compound']))
                        connection.commit()
        except (requests.exceptions.ConnectionError, requests.exceptions.ChunkedEncodingError) as err:
            #log_(err + ' Put to sleep before retrying.')
            time.sleep(100)
            continue   
    
     

In [12]:
def main():
    bearer_token = os.environ.get("BEARER_TOKEN")
    headers = create_headers(bearer_token)
    rules = get_rules(headers, bearer_token)
    delete = delete_all_rules(headers, bearer_token, rules)
    set = set_rules(headers, delete, bearer_token)
    get_stream(headers, set, bearer_token)


In [ ]:
if __name__ == "__main__":
    main()
